In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/game-of-thrones-books/003ssb.txt
/kaggle/input/game-of-thrones-books/001ssb.txt
/kaggle/input/game-of-thrones-books/004ssb.txt
/kaggle/input/game-of-thrones-books/005ssb.txt
/kaggle/input/game-of-thrones-books/002ssb.txt


In [2]:
df = pd.read_fwf('/kaggle/input/game-of-thrones-books/002ssb.txt',sep='\s+', index_col=False,columns=['Index'])

In [3]:
df.columns

Index(['A Clash of Kings'], dtype='object')

In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [5]:
sent_array=df.loc[0:500].to_numpy()

In [6]:
len(sent_array)

501

In [7]:
df1=pd.DataFrame(data=sent_array.flatten(),columns=['Sentences'])

In [8]:
df1['Sentences'][3]

"The comet's tail spread across the dawn, a red slash that bled above the crags of Dragonstone like a"

Since there is no classification , let's have a look at POS Tagging.

In [9]:
from nltk.tokenize import word_tokenize
import re

def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = word_tokenize(string)

    return string 

In [10]:
text_window=2
all_text=[]
word_lists=[]
for text in df1['Sentences']:
    text = clean_text(text)
    

    # Appending to the all text list
    all_text += text 

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(text_window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
                
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

In [11]:
def create_dict(text):
    word_list=list(set(text))
    word_list.sort()
    word_freq={};
    for i,word in enumerate(word_list):
        word_freq.update({
            word:i
        })
        
    return word_freq;

In [12]:
word_freq=create_dict(all_text)
word_lists

[['book', 'two'],
 ['book', 'of'],
 ['two', 'of'],
 ['two', 'book'],
 ['two', 'song'],
 ['of', 'song'],
 ['of', 'two'],
 ['of', 'of'],
 ['of', 'book'],
 ['song', 'of'],
 ['song', 'of'],
 ['song', 'ice'],
 ['song', 'two'],
 ['of', 'ice'],
 ['of', 'song'],
 ['of', 'fire'],
 ['of', 'of'],
 ['ice', 'fire'],
 ['ice', 'of'],
 ['ice', 'song'],
 ['fire', 'ice'],
 ['fire', 'of'],
 ['by', 'george'],
 ['by', 'r'],
 ['george', 'r'],
 ['george', 'by'],
 ['george', 'r'],
 ['r', 'r'],
 ['r', 'george'],
 ['r', 'martin'],
 ['r', 'by'],
 ['r', 'martin'],
 ['r', 'r'],
 ['r', 'george'],
 ['martin', 'r'],
 ['martin', 'r'],
 ['comets', 'tail'],
 ['comets', 'spread'],
 ['tail', 'spread'],
 ['tail', 'comets'],
 ['tail', 'across'],
 ['spread', 'across'],
 ['spread', 'tail'],
 ['spread', 'dawn'],
 ['spread', 'comets'],
 ['across', 'dawn'],
 ['across', 'spread'],
 ['across', 'red'],
 ['across', 'tail'],
 ['dawn', 'red'],
 ['dawn', 'across'],
 ['dawn', 'slash'],
 ['dawn', 'spread'],
 ['red', 'slash'],
 ['red', 'd

In [13]:
from scipy import sparse
from tqdm import tqdm
X=[]
Y=[]
num_words=len(word_freq)
words=word_freq.keys()

for i,context_word in tqdm(enumerate(word_lists)):
    main_word_index=word_freq.get(context_word[0])
    context_word_index=word_freq.get(context_word[1])
    
    X_row=np.zeros(num_words)
    Y_row=np.zeros(num_words)
    
    X_row[main_word_index]=1
    Y_row[context_word_index]=1
    
    X.append(X_row)
    Y.append(Y_row)
    
X = np.asarray(X)
Y = np.asarray(Y)

24820it [00:00, 39082.64it/s]


In [14]:
from keras.models import Input, Model
from keras.layers import Dense
import matplotlib.pyplot as plt

In [15]:
embed_size = 2
import itertools
# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [16]:
model.fit(
    x=X, 
    y=Y, 
    batch_size=32,
    epochs=500,
    )

Epoch 1/500
776/776 [==============================] - 2s 2ms/step - loss: 7.1007
Epoch 2/500
776/776 [==============================] - 2s 2ms/step - loss: 6.4522
Epoch 3/500
776/776 [==============================] - 2s 2ms/step - loss: 6.4006
Epoch 4/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3889
Epoch 5/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3818
Epoch 6/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3767
Epoch 7/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3727
Epoch 8/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3692
Epoch 9/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3661
Epoch 10/500
776/776 [==============================] - 2s 3ms/step - loss: 6.3633
Epoch 11/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3606
Epoch 12/500
776/776 [==============================] - 2s 2ms/step - loss: 6.3583
Epoch 13/500


776/776 [==============================] - 2s 2ms/step - loss: 6.0393
Epoch 100/500
776/776 [==============================] - 1s 2ms/step - loss: 6.0383
Epoch 101/500
776/776 [==============================] - 1s 2ms/step - loss: 6.0371
Epoch 102/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0361
Epoch 103/500
776/776 [==============================] - 1s 2ms/step - loss: 6.0348
Epoch 104/500
776/776 [==============================] - 1s 2ms/step - loss: 6.0338
Epoch 105/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0327
Epoch 106/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0317
Epoch 107/500
776/776 [==============================] - 1s 2ms/step - loss: 6.0307
Epoch 108/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0296
Epoch 109/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0287
Epoch 110/500
776/776 [==============================] - 2s 2ms/step - loss: 6.0277
Epoch 

776/776 [==============================] - 2s 2ms/step - loss: 5.9620
Epoch 197/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9615
Epoch 198/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9610
Epoch 199/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9605
Epoch 200/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9600
Epoch 201/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9594
Epoch 202/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9590
Epoch 203/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9586
Epoch 204/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9580
Epoch 205/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9575
Epoch 206/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9571
Epoch 207/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9566
Epoch 

776/776 [==============================] - 2s 2ms/step - loss: 5.9307
Epoch 294/500
776/776 [==============================] - 1s 2ms/step - loss: 5.9306
Epoch 295/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9304
Epoch 296/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9302
Epoch 297/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9300
Epoch 298/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9298
Epoch 299/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9296
Epoch 300/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9294
Epoch 301/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9292
Epoch 302/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9290
Epoch 303/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9289
Epoch 304/500
776/776 [==============================] - 1s 2ms/step - loss: 5.9288
Epoch 

776/776 [==============================] - 2s 2ms/step - loss: 5.9186
Epoch 391/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9186
Epoch 392/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9184
Epoch 393/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9184
Epoch 394/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9182
Epoch 395/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9181
Epoch 396/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9181
Epoch 397/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9180
Epoch 398/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9179
Epoch 399/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9179
Epoch 400/500
776/776 [==============================] - 2s 3ms/step - loss: 5.9178
Epoch 401/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9177
Epoch 

776/776 [==============================] - 2s 2ms/step - loss: 5.9125
Epoch 488/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9125
Epoch 489/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9125
Epoch 490/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9123
Epoch 491/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9123
Epoch 492/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9123
Epoch 493/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9121
Epoch 494/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9122
Epoch 495/500
776/776 [==============================] - 1s 2ms/step - loss: 5.9121
Epoch 496/500
776/776 [==============================] - 1s 2ms/step - loss: 5.9119
Epoch 497/500
776/776 [==============================] - 2s 2ms/step - loss: 5.9121
Epoch 498/500
776/776 [==============================] - 1s 2ms/step - loss: 5.9120
Epoch 

In [17]:
weights = model.get_weights()[0]

In [19]:
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[word_freq.get(word)]
        })

In [23]:
plt.figure(figsize=(10, 10))
for word in list(word_freq.keys()):
    coord = embedding_dict.get(word)
    plt.scatter(coord[0], coord[1])
    plt.annotate(word, (coord[0], coord[1]))

AttributeError: 'dict' object has no attribute 'iloc'

<Figure size 720x720 with 0 Axes>